In [73]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [74]:
from state import get_state, queue
import os
import traci

In [75]:
from dotenv import load_dotenv
load_dotenv()


sumo_bin = os.getenv("SUMO")
sumo_gui_bin = os.getenv("SUMO-GUI")
simulConfig = os.getenv("SIMUL-CONFIG")

In [76]:
import numpy as np

from tensorflow.keras import Sequential, layers
from tensorflow.random import set_seed
from collections import deque
from tensorflow import keras
from tensorflow import reduce_sum, reduce_mean, one_hot, GradientTape
from tensorflow.keras.losses import MeanSquaredError
import matplotlib.pyplot as plt
import tensorflow as tf

In [77]:
set_seed(42)  # extra code – ensures reproducibility on the CPU

input_shape = [48]  # == env.observation_space.shape
n_outputs = 4  # == env.action_space.n
#[((Phase(duration=30.0, state='GGrGrrGGrGrr', minDur=30.0, maxDur=30.0), Phase(duration=20.0, state='grGgrrgrGgrr', minDur=20.0, maxDur=20.0), Phase(duration=30.0, state='GrrGGrGrrGGr', minDur=30.0, maxDur=30.0), Phase(duration=20.0, state='grrgrGgrrgrG', minDur=20.0, maxDur=20.0)), [0, 2, 4, 6])]
model_action = Sequential([
    layers.Dense(32,activation=tf.keras.layers.LeakyReLU(alpha=0.01), input_shape=input_shape),
    layers.Dense(32, activation=tf.keras.layers.LeakyReLU(alpha=0.01)),
    layers.Dense(n_outputs, activation= 'linear')])

In [78]:
def epsilon_greedy_policy(state, epsilon=0):
    if np.random.rand() < epsilon:
        return np.random.randint(n_outputs)  # random action
    else:
        Q_values = model_action.predict(state[np.newaxis], verbose=0)[0]
        print(f"state : {state}")
        print(f"Q_values : {Q_values}")
        return Q_values.argmax()  # optimal action according to the DQN

In [79]:
def sample_experiences(batch_size):
    indices = np.random.randint(len(replay_buffer), size=batch_size)
    batch = [replay_buffer[index] for index in indices]
    states, actions, rewards, next_states = [
        np.array([experience[field_index] for experience in batch])
        for field_index in range(4)
    ]
    return states, actions, rewards, next_states

In [80]:
batch_size = 32
discount_factor = 0.5
optimizer = keras.optimizers.Adam(learning_rate=0.05)
loss_fn = MeanSquaredError()

def training_step(batch_size):
    experiences = sample_experiences(batch_size)
    states, actions, rewards, next_states = experiences  # a changer
    next_Q_values = model_action.predict(next_states, verbose=0)
    max_next_Q_values = next_Q_values.max(axis=1)
    # runs = 1.0 - (dones | truncateds)  # episode is not done or truncated
    target_Q_values = rewards + discount_factor * max_next_Q_values
    target_Q_values = target_Q_values.reshape(-1, 1)
    mask = one_hot(actions, n_outputs)
    with GradientTape() as tape:
        all_Q_values = model_action(states)
        Q_values = reduce_sum(all_Q_values * mask, axis=1, keepdims=True)
        loss = reduce_mean(loss_fn(target_Q_values, Q_values))

    grads = tape.gradient(loss, model_action.trainable_variables)
    optimizer.apply_gradients(zip(grads, model_action.trainable_variables))

In [88]:
rewards = []
best_score = 0
reward = 0
total_reward = 0
list_values = []
weights = []
wait_times = []
replay_buffer = deque(maxlen=10000)
epoch = 50

for episode in range(epoch):
    #SEED
    seed_value = np.random.randint(1000)
    #seed_value = 41
    sumoCmd = [sumo_bin, "-c", simulConfig, "--start", "--seed", str(seed_value), '--no-warnings']

    if traci.isLoaded():
        traci.close()
    traci.start(sumoCmd)
    lane_ids =  traci.lane.getIDList()
    # print(lane_ids[0])

    trafic_light_ids = traci.trafficlight.getIDList()

    # state = np.array(queue(lane_ids))
    state=np.array(get_state(lane_ids))
    action=-1
    # print(state)
    wait_times.append(0)
    for step in range(80000): ## TO CHANGED
        epsilon = max(1 - episode / epoch, 0.01)

        if step%2000 == 0:
            #print(f"longeur du buffer :{len(replay_buffer)}")
            ########################################################################
            ##Calcul de la reward
            # next_state = np.array(queue(lane_ids))
            next_state = np.array(get_state(lane_ids))
            # reward = calculate_reward(values, reward, total_reward)[0]
            # reward=(np.sum(state)-np.sum(next_state))
            reward = np.sum(state[:24])- np.sum(next_state[:24])
            replay_buffer.append((state, action, reward, next_state))

            list_values.append(queue(lane_ids))
            #########################################################################
            state=next_state
            action = epsilon_greedy_policy(state, epsilon)
            #print("action", action)
            traci.trafficlight.setPhase(trafic_light_ids[0],2*action)

            if len(replay_buffer) >= batch_size*10:
                training_step(batch_size)
                # new_weights = model_action.get_weights()
                # weights.append(new_weights)
                #print(f"Episode {episode}: new weights = {new_weights}")
            # else:
            #     print(f"Episode {episode}: pas assez de données dans le replay buffer.")
            # print(values)
            # if list_values:
            #    # print(f'list values {list_values[-1]}')


        traci.simulationStep()
    print(f'episode : {episode}')
    traci.close()


 Retrying in 1 seconds


Step #720.00 (0ms ?*RT. ?UPS, TraCI: 0ms, vehicles TOT 786 ACT 32 BUF 0)                   #180.00 (1ms ~= 10.00*RT, ~93000.00UPS, TraCI: 3ms, vehicles TOT 230 ACT 93 BUF 4)   0.00 (0ms ?*RT. ?UPS, TraCI: 7ms, vehicles TOT 485 ACT 124 BUF 0)                  (0ms ?*RT. ?UPS, TraCI: 7ms, vehicles TOT 715 ACT 169 BUF 1)                 episode : 0
Step #800.00 (0ms ?*RT. ?UPS, TraCI: 1ms, vehicles TOT 786 ACT 4 BUF 0)                   
 Retrying in 1 seconds


Step #179.00 (0ms ?*RT. ?UPS, TraCI: 0ms, vehicles TOT 207 ACT 102 BUF 0)                 state : [ 0  0 14  0  9 12  0  0 12  0  8  8  0  0  0  0  0  0  0  0  0  0  0  0
  3  3 15  4 12 15  8  2 12  2  8  9  1  1  0  3  0  0  2  0  0  0  0  0]
Q_values : [-9.970377 -5.387272 -4.266041 -3.680519]
Step #720.00 (0ms ?*RT. ?UPS, TraCI: 0ms, vehicles TOT 703 ACT 4 BUF 0)                   0.00 (0ms ?*RT. ?UPS, TraCI: 7ms, vehicles TOT 408 ACT 112 BUF 0)                  (0ms ?*RT. ?UPS, TraCI: 7ms, vehicles TOT 624 ACT 107 BUF 0)                 episode : 1
Step #800.00 (0ms ?*RT. ?UPS, TraCI: 0ms, vehicles TOT 703 ACT 0 BUF 0)                   
 Retrying in 1 seconds


Step #540.00 (0ms ?*RT. ?UPS, TraCI: 0ms, vehicles TOT 645 ACT 118 BUF 0)                  #180.00 (0ms ?*RT. ?UPS, TraCI: 3ms, vehicles TOT 197 ACT 74 BUF 1)                  0.00 (1ms ~= 10.00*RT, ~124000.00UPS, TraCI: 6ms, vehicles TOT 440 ACT 124 BUF 7) state : [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0]
Q_values : [-1.2863379  -0.6454021  -0.65432364 -1.0530971 ]
Step #720.00 (0ms ?*RT. ?UPS, TraCI: 0ms, vehicles TOT 707 ACT 0 BUF 0)                   episode : 2
Step #800.00 (0ms ?*RT. ?UPS, TraCI: 0ms, vehicles TOT 707 ACT 0 BUF 0)                   
 Retrying in 1 seconds


Step #720.00 (0ms ?*RT. ?UPS, TraCI: 0ms, vehicles TOT 715 ACT 48 BUF 0)                   #180.00 (0ms ?*RT. ?UPS, TraCI: 7ms, vehicles TOT 211 ACT 116 BUF 2)                 0.00 (0ms ?*RT. ?UPS, TraCI: 3ms, vehicles TOT 420 ACT 94 BUF 3)                   (0ms ?*RT. ?UPS, TraCI: 4ms, vehicles TOT 627 ACT 99 BUF 3)                  episode : 3
Step #800.00 (0ms ?*RT. ?UPS, TraCI: 0ms, vehicles TOT 715 ACT 16 BUF 0)                  
 Retrying in 1 seconds


...

state : [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0]
Q_values : [-3.1155987 -3.6042824 -3.505831  -1.979195 ]
state : [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 2 1 2 3 1 4 1 0 3 3 0 0
 0 0 0 0 0 0 0 0 0 0 0]
Q_values : [-12.219868 -12.499163 -10.198603  -9.874588]
state : [0 2 1 0 1 0 0 1 0 0 2 0 0 0 0 0 0 0 0 0 0 0 0 0 2 3 3 5 3 1 2 2 2 3 4 1 1
 0 0 2 0 0 1 0 1 0 0 0]
Q_values : [ -9.198452 -10.011219  -8.493137  -8.935106]
state : [0 3 3 0 0 1 0 2 1 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 4 5 4 2 4 6 5 5 3 3 2 3 2
 0 0 0 3 0 2 0 1 1 3 0]
Q_values : [-8.422281  -8.683621  -9.916552  -7.1244745]
state : [0 4 4 0 2 0 0 5 2 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 2 5 7 6 6 2 2 9 4 5 3 1 2
 0 3 3 0 0 2 0 6 1 0 0]
Q_values : [-3.1497648 -5.209242  -6.2583804 -7.1779284]
state : [0 0 7 0 6 1 0 0 4 0 2 1 0 0 0 0 0 0 0 0 0 0 0 0 4 4 7 6 8 5 3 4 4 3 3 3 3
 7 1 0 0 0 2 3 2 2 0 0]
Q_values : [-6.1335335 -2.5499837 -4.4494133 -4.7796597]
state : [0 3 0 0

state : [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0]
Q_values : [-2.7745736 -3.7453933 -3.094846  -2.5863867]
state : [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 3 1 0 4 0 1 4 2 1 5 3 1 0
 0 0 0 0 0 0 0 0 0 0 0]
Q_values : [ -9.054026 -11.620963  -9.072876  -9.454142]
state : [0 0 0 0 0 1 0 0 1 0 1 1 0 0 0 0 0 0 0 0 0 0 0 0 7 3 3 3 2 1 3 0 1 6 3 3 2
 2 0 2 0 0 1 0 0 1 0 0]
Q_values : [-7.4237313 -8.961319  -8.221844  -7.300393 ]
state : [0 1 1 0 1 0 0 0 1 0 2 0 0 0 0 0 0 0 0 0 0 0 0 0 6 5 3 4 4 1 2 3 2 2 9 2 3
 0 3 1 0 0 4 0 1 5 0 0]
Q_values : [-11.779754 -15.398945 -10.217685 -14.02313 ]
state : [0 5 3 0 0 1 0 1 2 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 5 7 4 7 3 4 5 3 5 4 0 6 1
 0 0 1 5 1 2 0 0 3 3 0]
Q_values : [ -6.3369007  -9.564833  -10.474552   -6.7394176]
state : [0 0 4 0 1 3 0 0 4 0 0 6 0 0 0 0 0 0 0 0 0 0 0 0 5 2 6 5 9 5 4 5 9 6 2 7 4
 1 1 2 0 0 2 6 1 3 0 0]
Q_values : [-10.569316  -9.20468   -9.479128  -5.615142]
state : [ 0 

state : [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0]
Q_values : [-2.414886  -2.8222814 -2.8048458 -1.8246534]
state : [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 3 0 1 3 2 0 6 2 2 0
 0 0 0 0 0 0 0 0 0 0 0]
Q_values : [-8.801778 -8.441262 -9.275198 -7.556676]
state : [0 1 1 0 1 0 0 0 1 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 3 3 3 2 1 0 4 3 3 5 6 3 1
 0 1 0 0 0 2 0 1 0 0 0]
Q_values : [-11.12399   -8.710468  -9.5912   -10.02558 ]
state : [ 0  1  0  0  1  0  0  2  0  0  5  3  0  0  0  0  0  0  0  0  0  0  0  0
  2  2  2  4  5  0  1  6  1  4 10  4  1  0  1  4  0  3  5  0  0  1  0  3]
Q_values : [-7.1076384 -9.858889  -3.6853347 -6.895349 ]
state : [0 2 2 0 0 0 0 4 1 0 0 4 0 0 0 0 0 0 0 0 0 0 0 0 2 6 4 2 4 5 6 8 3 2 2 8 2
 0 0 0 9 1 2 0 0 0 3 0]
Q_values : [-10.464136 -10.444897 -13.09949   -7.424513]
state : [ 0  3  4  0  1  0  0  7  1  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  1 10  5  2  6  3  8 11  6  3  5  3  1  0  8  3  0  0 

state : [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0]
Q_values : [-2.676548  -2.4878721 -3.0643673 -2.3081954]
state : [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 2 1 0 5 3 0 3 1 0 3 3 1 0
 0 0 0 0 0 0 0 0 0 0 0]
Q_values : [-11.114915  -9.671401 -10.530094 -10.602102]
state : [0 0 0 0 1 0 0 0 0 0 1 1 0 0 0 0 0 0 0 0 0 0 0 0 5 3 0 5 6 1 3 3 0 4 4 2 1
 0 0 0 0 0 0 0 0 0 0 0]
Q_values : [-12.519873 -11.109139 -11.168244 -12.466236]
state : [0 2 0 0 5 1 0 2 0 0 4 2 0 0 0 0 0 0 0 0 0 0 0 0 4 6 2 7 9 4 4 5 2 3 6 4 3
 0 0 2 0 0 2 0 0 3 0 0]
Q_values : [-8.424225 -9.076491 -5.134228 -9.826292]
state : [ 0  4  2  0  0  4  0  4  2  0  0  4  0  0  0  0  0  0  0  0  0  0  0  0
  3  7  4  2  2  9  6 10  4  5  2  5  4  0  0  1  5  0  1  0  0  2  7  0]
Q_values : [ -4.8311853  -2.684461  -10.928347   -3.3963456]
state : [ 0  7  0  0  1  9  0  8  0  0  1  5  0  0  0  0  0  0  0  0  0  0  0  0
  5  8  1  5  5 11  6 12  4  6  3  6  1  0  0  3  0

state : [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0]
Q_values : [-2.2470841 -2.8597608 -2.795867  -2.086863 ]
state : [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 2 1 0 3 1 1 1 4 1 6 3 0 0
 0 0 0 0 0 0 0 0 0 0 0]
Q_values : [-10.400682 -10.86664   -9.929914 -10.486454]
state : [0 0 3 0 0 3 0 0 2 0 1 2 0 0 0 0 0 0 0 0 0 0 0 0 7 2 9 4 1 7 3 1 3 5 1 5 0
 4 0 2 0 0 5 1 0 3 0 0]
Q_values : [-7.5799594 -2.9026923 -6.959626  -4.6924386]
state : [0 0 0 0 0 6 0 1 0 0 1 5 0 0 0 0 0 0 0 0 0 0 0 0 5 5 1 6 2 9 1 3 2 3 3 8 2
 0 0 2 0 7 3 0 0 4 0 3]
Q_values : [-7.682519  -7.300917  -7.7950754 -1.92875  ]
state : [0 3 1 0 1 0 0 3 2 0 3 0 0 0 0 0 0 0 0 0 0 0 0 0 4 6 3 2 9 0 1 4 3 1 5 0 4
 0 8 0 0 1 2 0 9 4 0 1]
Q_values : [-3.8443756 -4.749043  -3.9259634 -8.246258 ]
state : [0 0 3 0 7 0 0 0 3 0 4 0 0 0 0 0 0 0 0 0 0 0 0 0 4 1 5 5 9 2 2 3 4 3 8 2 2
 3 0 1 0 0 2 4 1 3 0 0]
Q_values : [-7.813082  -6.714816  -2.4277265 -6.7144923]
state : [0 1 5 0

In [92]:
rewards = []
best_score = 0
reward = 0
total_reward = 0
list_values = []
weights = []
wait_times = []
# replay_buffer = deque(maxlen=2000)

#SEED
seed_value = np.random.randint(1000)
#seed_value = 41
sumoCmd = [sumo_gui_bin, "-c", simulConfig, "--start", "--seed", str(seed_value)]

if traci.isLoaded():
    traci.close()
traci.start(sumoCmd)
lane_ids =  traci.lane.getIDList()
# print(lane_ids[0])
# for lane in lane_ids:
#     print(traci.lane.getLastStepVehicleNumber(lane))
# north_lane = traci.lane.getLastStepVehicleNumber("N_0")
# south_lane = traci.lane.getLastStepVehicleNumber("S_0")
# east_lane = traci.lane.getLastStepVehicleNumber("E_0")
# west_lane = traci.lane.getLastStepVehicleNumber("W_0")
trafic_light_ids = traci.trafficlight.getIDList()

state = np.array(get_state(lane_ids))
action=-1
# print(state)
wait_times.append(0)
for step in range(100000): ## TO CHANGED
    #epsilon = max(1 - episode / 10, 0.01)
    nom_du_feu= traci.trafficlight.getIDList()[0]

    if step%2000 == 0:
        state=np.array(get_state(lane_ids))
        action = epsilon_greedy_policy(state,0)*2
        # action = np.random.randint(8)
        print("action", action)
        #print(traci.trafficlight.getAllProgramLogics(nom_du_feu))

        #print(traci.trafficlight.getAllProgramLogics(nom_du_feu)[0].phases[action])
        traci.trafficlight.setPhase(trafic_light_ids[0],action)
    traci.simulationStep()

traci.close()


 Retrying in 1 seconds
state : [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0]
Q_values : [-1.9985943 -2.986529  -2.6044343 -1.9641954]
action 6
state : [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 4 5 3 6 3 3 5 6 3 5 2 0 0
 0 0 0 0 0 0 0 0 0 0 0]
Q_values : [-12.212148  -15.789782  -12.175854  -15.5898485]
action 4
state : [ 0  1  1  0  0  1  0  2  2  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  4  9  9  4  8  7  5 12  6  6  3  1  1  0  0  0  1  0  2  0  0  1  0  0]
Q_values : [-15.527282 -19.307728 -15.268804 -21.734404]
action 4
state : [ 0  8  7  1  0  6  0  8  6  0  0  1  0  0  0  0  0  0  0  0  0  0  0  0
  3 12 11  7  6  9  5 16  8  7  4  1  1  0  0  2  2  1  2  0  0  0  5  0]
Q_values : [ 5.564353   5.2033463 -9.127427  -4.615077 ]
action 0
state : [ 0  0 11  0  1  9  0  0  7  1  3  1  0  0  0  0  0  0  0  0  0  0  0  0
  5  6 15  6  6 10  5  9 10  9  4  5  4  5  4  3  0  0  5  5  3  2  0  0]
Q_values : [-5.9652185  3.7164

FatalTraCIError: Connection closed by SUMO.